In [ ]:
#-----#
#Used for nested sweeping of Keithley SMUs, for recording I,V from Keithleys and Nanovoltmeters
#Using QCodes for data storage, 
#Use plottr-inspectr for real-time plotting (open separately)

#Created by Lewis P. and Eli H
#Updated 16/10/2025



In [ ]:
#just imports helper functions software
from import_all import *
from utilities import *


In [ ]:
#----Connect to instrument----#
#load the station from yaml configuration file
station = Station(config_file="electrochemistry.station.yaml")
keithley1 = station.load_keithley1()
keithley2 = station.load_keithley2()
#nano1 = station.load_nanovoltmeter1()
#nano2 = station.load_nanovoltmeter2()

In [ ]:
#---Save paths----#
csv_save_path = "../csv/test/"
db_save_path = "../db/test/"
device_name = "test" #master database file
xpmnt_name = "test"#xpmnt name within database. Saved as seperate csv

In [ ]:
####################################
#----Configure channels----#
####################################

ch1 = { "channel": keithley1.smua,
        "name": "top_gate", 
        "first_node": 0.005, # from start -> first_node -> second_node -> start
        "second_node": -0.005,
        "start_voltage": 0,
        "dV": 0.0002, #voltage step
        "independent": True,  #independent variable for plotting purposes? Can be buggy if more than one independent variable
        "manual": False,
        "manual_vRange": [],
               
}


ch2 = { "channel": keithley1.smub,
        "name": "bot_gate",
        "first_node": 0, # from start -> first_node -> second_node -> start
        "second_node": 0,
        "start_voltage": 0,
        "dV": 0.00, #voltage step
        "independent": False,  #independent variable for plotting purposes? Can be buggy if more than one independent variable
        "manual": False,
        "manual_vRange": []
}

ch3 = { "channel": keithley2.smub,
        "name": "drain_source",
        "first_node":0.00,
        "second_node": 0.00,
        "start_voltage":0.00,
        "dV": 0.0,
        "independent":  False,
        "manual": False,
        "manual_vRange": []

}

ch4 = { "channel": keithley2.smua,
        "name": "temperature", #handled differently. If measuring temperature, make sure to include "temperature" in name
        "first_node":5e-3,
        "second_node": 5e-3,
        "start_voltage":5e-3,
        "dV": 0,
        "independent":  False,
        "manual": False,
        "manual_vRange": []

}

#ch5 = { "channel": nano1,
        # "name": "nano1_tg",
        # "first_node":0,
        # "second_node": 0,
        # "start_voltage":0,
        # "dV": 0,
        # "independent":  False,
        # "manual": False,
        # "manual_vRange": []

#  }
#ch6 = { "channel": nano2,
#         "name": "nano2_bg",
#         "first_node":0,
#         "second_node": 0,
#         "start_voltage":0,
#         "dV": 0,
#         "independent":  False,
#         "manual": False,
#         "manual_vRange": []

# }

####################################
#----Configure sweeping params----#
####################################
time_independent = False  #plot time as indepent variable?

ramp_up = True # ramp to initial values?
ramp_down = False #ramp back to 0?

temp_measure = True

ktime_control = True

dt_list = [1] #time per point in seconds

delayNPLC_ratio = 0.8 #ratio of delay to total time dt.

start_delay = 1
write_period = 2
repeat= 1
round_delay= 0

#Sweep order determines hierarchy of sweeping: Sweep 1st element; 
#for every 1st element, sweep second element; for every second element, sweep third element
sweepers = [ch1,ch2,ch3,ch4]#,ch4,ch5,ch6]
sweepers_save_order = [ch1, ch2, ch3, ch4]#, ch4,ch5,ch6] #Order for saving in database, for easy parsing of data in plottr-inspectr
n_measurements = len(sweepers)

In [ ]:
###### Perform checks ######
i = 0
for sweeper in sweepers:
    if sweeper["independent"]:
        i+=1

if time_independent:
    i+=1
if not i == 1:
    print("-- Greater or fewer than 1 channel is independent. Plottr-inspectr my not function correctly")

if not ramp_up:
    print("-- Voltages will start instaneously at initial values. Are you sure you don't want to ramp?")


if not ktime_control:
    print("-- Measurement time on Keithley is not controlled. Make sure appropriate speeds are set manually")


###### Print scan speeds ########
for sweeper in sweepers:
    print(f'----{sweeper["name"]}----')
    if sweeper["dV"] ==0:
        print(f'Keeping {sweeper["name"]} fixed at {sweeper["second_node"]} V')
        continue

    print(f'Will scan {sweeper["name"]} from {sweeper["first_node"]} to {sweeper["second_node"]} in {1000*sweeper["dV"]} mV steps')   
    print(f'Starting and ending at {sweeper["start_voltage"]}')
    print(f'Scan speed for {sweeper["name"]}: {1000 * sweeper["dV"]/dt} mV/s')
    print(f'Number of points for {sweeper["name"]}: {1 + abs(int((sweeper["second_node"]-sweeper["first_node"])/sweeper["dV"]))} steps')

In [ ]:
#####set scan lists and plot
i = 0
fig = plt.figure(figsize=(20, 3))
for sweeper in sweepers:
    startvoltage = sweeper["start_voltage"]
    firstnode = sweeper["first_node"]
    secondnode = sweeper["second_node"]
    dV = sweeper["dV"]

    if dV == 0:
        sweeper["v_range"] = [startvoltage]
    else:
        n = 1+abs(int((firstnode-startvoltage)/dV))
        v_range1 = np.linspace(startvoltage,firstnode,n)
        v_range1 = v_range1[:-1] #remove duplicate point

        n = 1+abs(int((secondnode-firstnode)/dV))
        v_range2 = np.linspace(firstnode,secondnode,n)
        v_range2 = v_range2[:-1] #remove duplicate point

        n = 1+abs(int((startvoltage-secondnode)/dV))
        v_range3 = np.linspace(secondnode,startvoltage,n)
    
        sweeper["v_range"] = np.concatenate((v_range1,v_range2, v_range3))    

    if sweeper["manual"]:
        sweeper["v_range"] = sweeper["manual_vRange"]


    ax = fig.add_subplot(1,len(sweepers),i+1)
    ax.plot(sweeper["v_range"], marker = '.')
    ax.set_title(sweeper["name"])
    i+=1

fig.tight_layout()

In [ ]:
def set_ktime(dt_in = dt_list[0], ktime_control = True):
    if not ktime_control:
        return
    nplc_set = dt_in*50*(1-delayNPLC_ratio)/(n_measurements)
    delay = dt_in - (nplc_set/50)*n_measurements

    sweepers_save_order[0]["channel"].delay(delay)
    sweepers_save_order[0]["channel"].nplc(nplc_set)

    for sweeper in sweepers_save_order[1:]:
        if not "nano" in sweeper["name"]:
            sweeper["channel"].delay(0)
        sweeper["channel"].nplc(nplc_set)

In [ ]:
set_ktime(dt_list[0],ktime_control=ktime_control)

In [ ]:
#----Connect to database----#
initialise_or_create_database_at(f'{db_save_path}{device_name}_arbSweeper.db')
#Set up experiment object
test_exp = load_or_create_experiment(
    experiment_name=xpmnt_name,
    sample_name=device_name,
)

meas_forward, time, independent_params = setup_database_registers_arb(station, test_exp, sweepers_save_order, time_independent=time_independent)
meas_forward.write_period = write_period


In [ ]:
#ramp voltages to initial values, from 0
#initial values: 
if ramp_up:
    for sweeper in sweepers:
        if not "nano" in sweeper["name"]:
            print(f'ramping {sweeper["name"]} to {sweeper["v_range"][0]}')
            ramp_voltage(sweeper["channel"], sweeper["v_range"][0])

print(f'Delay for {start_delay} s...')
sleep(start_delay)
print("Starting sweeps")
time.reset_clock()

v_ranges = [d["v_range"] for d in sweepers]
with meas_forward.run() as forward_saver:
    for dt_in in dt_list:
        set_ktime(dt_in,ktime_control)
        for rep in range(repeat):
            print(f'Repeat: {rep}')
            first_sweep = True            
            for v_range in itertools.product(*v_ranges):
                for x,sweeper in zip(v_range,sweepers):
                    if not "nano" in sweeper["name"]:
                        sweeper["channel"].volt(x)
                        sweeper["voltage"] = x

                if rep >  0 and first_sweep:
                        first_sweep = False    
                        continue
                
                t = time()
                get_readings = []
                independent_params = []
                
                if not ktime_control:
                    sleep(dt_in)

                for sweeper in sweepers_save_order:
                    if "nano" in sweeper["name"]:
                        v = sweeper["channel"].volt()
                    else:
                        v = sweeper["voltage"]
                        j = sweeper["channel"].curr()
                        get_readings.append((sweeper["channel"].curr, j))

                    if sweeper["independent"]:
                        independent_params.append((sweeper["channel"].volt, v))

                    if not sweeper["independent"]:
                        get_readings.append((sweeper["channel"].volt, v))

                    if "temperature" in sweeper["name"]:
                        temperature = rToT(v/j)
                        get_readings.append((sweeper["channel"].temperature, temperature))

                    

                forward_saver.add_result(
                    *independent_params,
                    *get_readings,
                    (time, t)
                    )
                
data_forward = forward_saver.dataset

#save data to csv
data_forward.to_pandas_dataframe().to_csv(f"{csv_save_path}{device_name}{xpmnt_name}_{data_forward.run_id}_manual_sweep.csv")

# Ramping voltage back to 0

if ramp_down:
    for sweeper in sweepers:
        if not "nano" in sweeper["name"]:
            print(f'ramping {sweeper["name"]} to 0')
            ramp_voltage(sweeper["channel"], 0)